# DATA WRANGLING PROJECT
Calculate the total revenue of each product at relevant price using 2 datasets
- prices.csv: records of product's history price changes in Sep 2018
- sales.csv: records of product's sales in Sep 2018

In [1]:
import pandas as pd
sales = pd.read_csv('4. Python_Data_sales.csv')
prices = pd.read_csv('4. Python_Data_prices.csv')

In [2]:
# Convert datetime
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'], format = '%m/%d/%y %H:%M')
prices['updated_at'] = pd.to_datetime(prices['updated_at'], format = '%m/%d/%y %H:%M')

In [3]:
# Sort 2 df
sales = sales.sort_values('ordered_at')
prices = prices.sort_values('updated_at')

In [4]:
print(sales.head(10))
print(prices.head(10))

     product_id          ordered_at  quantity_ordered
86      3954203 2018-09-11 01:43:00                 1
28      4085861 2018-09-11 06:26:00                 1
26      4085861 2018-09-11 06:53:00                 1
27      4085861 2018-09-11 08:24:00                 1
123     4085861 2018-09-11 09:30:00                 1
113     4085861 2018-09-11 11:06:00                 1
25      3954203 2018-09-11 11:11:00                 1
24      3954203 2018-09-11 11:11:00                 1
112     4085861 2018-09-11 11:34:00                 1
107     4085861 2018-09-11 11:47:00                 2
    product_id  old_price  new_price          updated_at
4      3954203      68800      60000 2018-09-10 16:32:00
7      3998909      19000      17000 2018-09-10 16:35:00
0           64     270000     239000 2018-09-10 16:37:00
11     4085861      60000      53500 2018-09-11 08:51:00
1      3954203      60000      64000 2018-09-11 11:54:00
9      4085861      53500      67000 2018-09-12 03:51:00
6      

In [5]:
# Merge sales and price table backward using merge_asof 
merge_back = pd.merge_asof(sales, prices, left_on='ordered_at', right_on='updated_at', by='product_id')
merge_back = merge_back[['product_id', 'ordered_at', 'quantity_ordered', 'new_price']].rename(columns={'new_price':'price'})
print(merge_back)

     product_id          ordered_at  quantity_ordered    price
0       3954203 2018-09-11 01:43:00                 1  60000.0
1       4085861 2018-09-11 06:26:00                 1      NaN
2       4085861 2018-09-11 06:53:00                 1      NaN
3       4085861 2018-09-11 08:24:00                 1      NaN
4       4085861 2018-09-11 09:30:00                 1  53500.0
..          ...                 ...               ...      ...
170     4085861 2018-09-18 20:23:00                 1  52000.0
171     4085861 2018-09-18 20:43:00                 1  52000.0
172     4085861 2018-09-18 20:54:00                 1  52000.0
173     3954203 2018-09-18 21:26:00                 1  57500.0
174     3998909 2018-09-18 22:11:00                 1  16500.0

[175 rows x 4 columns]


In [6]:
# Merge merge_back and price table forward using merge_asof 
merge_forw = pd.merge_asof(merge_back, prices, left_on='ordered_at', right_on='updated_at', by='product_id', 
                           direction='forward')
print(merge_forw.head())

   product_id          ordered_at  quantity_ordered    price  old_price  \
0     3954203 2018-09-11 01:43:00                 1  60000.0    60000.0   
1     4085861 2018-09-11 06:26:00                 1      NaN    60000.0   
2     4085861 2018-09-11 06:53:00                 1      NaN    60000.0   
3     4085861 2018-09-11 08:24:00                 1      NaN    60000.0   
4     4085861 2018-09-11 09:30:00                 1  53500.0    53500.0   

   new_price          updated_at  
0    64000.0 2018-09-11 11:54:00  
1    53500.0 2018-09-11 08:51:00  
2    53500.0 2018-09-11 08:51:00  
3    53500.0 2018-09-11 08:51:00  
4    67000.0 2018-09-12 03:51:00  


In [7]:
# Fill  null values w old_price
merge_forw['price'] = merge_forw['price'].fillna(merge_forw['old_price'])
sales_w_prices = merge_forw[['product_id', 'ordered_at', 'quantity_ordered', 'price']]
print(sales_w_prices)

     product_id          ordered_at  quantity_ordered    price
0       3954203 2018-09-11 01:43:00                 1  60000.0
1       4085861 2018-09-11 06:26:00                 1  60000.0
2       4085861 2018-09-11 06:53:00                 1  60000.0
3       4085861 2018-09-11 08:24:00                 1  60000.0
4       4085861 2018-09-11 09:30:00                 1  53500.0
..          ...                 ...               ...      ...
170     4085861 2018-09-18 20:23:00                 1  52000.0
171     4085861 2018-09-18 20:43:00                 1  52000.0
172     4085861 2018-09-18 20:54:00                 1  52000.0
173     3954203 2018-09-18 21:26:00                 1  57500.0
174     3998909 2018-09-18 22:11:00                 1  16500.0

[175 rows x 4 columns]


In [8]:
# Calculate individual order's revenue
sales_w_prices['revenue'] = sales_w_prices.quantity_ordered * sales_w_prices.price

# Calculate total revenue
total_revenue = sales_w_prices.groupby('product_id').agg({'revenue':'sum'})

/var/folders/3q/ncj57g4935l266gmcj7_bvs40000gn/T/ipykernel_1936/2335701143.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_w_prices['revenue'] = sales_w_prices.quantity_ordered * sales_w_prices.price


In [9]:
total_revenue

,revenue
product_id,
64,956000.0
3954203,877500.0
3998909,280500.0
4085861,8247500.0
